<img src=./imgs/model_io.jpg width=35% />

[langchain documents](https://python.langchain.com/docs/modules/model_io/models/chat/llm_chain)

[LangChain-Tutorials](https://github.com/sugarforever/LangChain-Tutorials)

> 根据Embedding的余弦相似度选择近似的示例 作为prompt模板的输入

In [1]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings

> embeddings = self._embedding_function.embed_documents(list(texts))   都是调用的 embed_documents

In [3]:
model_name = "sentence-transformers/all-mpnet-base-v2"
# 效果不好, 是感觉,所以应该是 happy sad 
#Give the antonym of every input
#
#Input: energetic: 精力充沛
#Output: lethargic
#
#Input: worried
#Output:
# model_name = "/home/10170464/jupyter-notebook/Wangquanjun/langchain-ChatGLM/huggingface_tools/dataroot/models/GanymedeNil/text2vec-large-chinese"

# model_name = "/home/10170464/jupyter-notebook/Wangquanjun/all-mpnet-base-v2"
# 这个效果好一些, 以后可以考虑使用这个
#Give the antonym of every input
#
#Input: happy
#Output: sad
#
#Input: worried
#Output:

model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
emb_ins = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

ConnectionError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/models/sentence-transformers/all-mpnet-base-v2 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd6f8048520>: Failed to establish a new connection: [Errno 101] Network is unreachable'))"), '(Request ID: 88b09a2c-daa0-4bcd-9917-40baa026f752)')

In [4]:
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

In [5]:
# Examples of a pretend task of creating antonyms
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

In [6]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity
    emb_ins,
    # The VectorStore class that is used to store the embeddings and do a mililarity search over.
    Chroma,
    # The number of examples to produce
    k=1
)

In [7]:
similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:", 
    input_variables=["adjective"],
)

In [8]:
# Input is a feeling, so should select the happy/sad example
print(similar_prompt.format(adjective='worried'))

Give the antonym of every input

Input: happy
Output: sad

Input: worried
Output:


In [9]:
# Input is a measurement, so should select the tall/short example
print(similar_prompt.format(adjective="large"))

Give the antonym of every input

Input: tall
Output: short

Input: large
Output:


In [10]:
# You can add new examples to the SemanticSimilarityExampleSelector as well.
similar_prompt.example_selector.add_example({"input": "enthusiastic", "output": "apathetic"})
print(similar_prompt.format(adjective='passionate'))

Give the antonym of every input

Input: enthusiastic
Output: apathetic

Input: passionate
Output:
